# [TensorBoard Tutorial](https://www.datacamp.com/community/tutorials/tensorboard-tutorial)

## Initial setup

In [1]:
# Load packages.
import urllib.request, json
import os

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import datetime as dt
import tensorflow as tf

from pandas_datareader import data
from tensorflow.examples.tutorials.mnist import input_data

## Starting TensorBoard

## Understanding the Benefits of Scalar Visualization

In [2]:
def accuracy(predictions, labels):
    """
    Accuracay of a given set of predictions of size (N x n_classes) and labels of size 
    (N x n_classes).
    """
    return np.sum(
        np.argmax(predictions, axis=1) == np.argmax(labels, axis=1) * \
        100.0 / labels.shape[0]
    )

## Define Inputs, Outputs, Weights and Biases

In [3]:
batch_size = 100
layer_ids = ["hidden1", "hidden2", "hidden3", "hidden4", "hidden5", "out"]
layer_sizes = [784, 500, 400, 300, 200, 100, 10]

tf.reset_default_graph()

# Inputs and labels.
train_inputs = tf.placeholder(tf.float32, 
                              shape=(batch_size, layer_sizes[0]), 
                              name="train_inputs")

train_labels = tf.placeholder(tf.float32, 
                              shape=(batch_size, layer_sizes[-1]),
                              name="train_labels")

# Weight and bias definitions.
for idx, lid in enumerate(layer_ids):

    with tf.variable_scope(lid):
        w = tf.get_variable("weights", 
                            shape=(layer_sizes[idx], layer_sizes[idx + 1]),
                            initializer=tf.truncated_normal_initializer(stddev=0.05))
        b = tf.get_variable("bias",
                            shape=(layer_sizes[idx+1]),
                            initializer=tf.random_uniform_initializer(-0.1, 0.1))

## Calculating Logits, Predictions, Loss and Optimization

In [4]:
# Calculating logits.
h = train_inputs

for lid in layer_ids:
    with tf.variable_scope(lid, reuse=True):
        
        w, b = tf.get_variable("weights"), tf.get_variable("bias")
        
        if lid != "out":
            h = tf.nn.relu(tf.matmul(h, w) + b, name=lid + "_output")
        else:
            h = tf.nn.xw_plus_b(h, w, b, name=lid + "_output")

tf_predictions = tf.nn.softmax(h, name='predictions')

# Calculating loss.
tf_loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits_v2(labels=train_labels, logits=h), name="loss"
)

# Optimizer.
tf_learning_rate = tf.placeholder(tf.float32, shape=None, name="learning_rate")
optimizer = tf.train.MomentumOptimizer(tf_learning_rate, momentum=0.9)
grads_and_vars = optimizer.compute_gradients(tf_loss)
tf_loss_minimize = optimizer.minimize(tf_loss)

## Defining Summaries

In [5]:
# Name scope allows you to group various summaries together. Summaries having the same 
# name_scope will be displayed on the same row.
with tf.name_scope("performance"):
    
    # Summaries need to be displayed whenever you need to record the loss, feed the mean loss
    # to this placeholder.
    tf_loss_ph = tf.placeholder(tf.float32, shape=None, name="loss_summary")
    
    # Create a scalar summary object for the loss so it can be displayed.
    tf_loss_summary = tf.summary.scalar("loss", tf_loss_ph)

    # Whenever you need to record the loss, feed the mean test accuracy to this placeholder.
    tf_accuracy_ph = tf.placeholder(tf.float32, shape=None, name="accuracy_summary")
    
    # Create a scalar summary object for the accuracy so it can be displayed.
    tf_accuracy_summary = tf.summary.scalar("accuracy", tf_accuracy_ph)

# Gradient norm summary.
for g, v in grads_and_vars:
    if "hidden5" in v.name and "weights" in v.name:
        
        with tf.name_scope("gradients"):
            tf_last_grad_norm = tf.sqrt(tf.reduce_mean(g ** 2))
            tf_gradnorm_summary = tf.summary.scalar("grad_norm", tf_last_grad_norm)
            break

# Merge all summaries together.
performance_summaries = tf.summary.merge([tf_loss_summary, tf_accuracy_summary])

## Executing the neural network: Loading Data, Training, Validation and Testing

In [6]:
image_size = 28
n_channels = 1
n_classes = 10
n_train = 55000
n_valid = 5000
n_test = 10000
n_epochs = 25

config = tf.ConfigProto(allow_soft_placement=True)
config.gpu_options.allow_growth = True
config.gpu_options.per_process_gpu_memory_fraction = 0.9 # Making sure Tensorflow doesn't 
                                                         # overflow the GPU.
session = tf.InteractiveSession(config=config)

if not os.path.exists("summaries"):
    os.mkdir("summaries")

if not os.path.exists(os.path.join("summaries", "second")):
    os.mkdir(os.path.join("summaries", "second"))

summ_writer = tf.summary.FileWriter(os.path.join("summaries", "second"), session.graph)

tf.global_variables_initializer().run()

accuracy_per_epoch = []
mnist_data = input_data.read_data_sets("MNIST_data", one_hot=True)

for epoch in range(n_epochs):
    loss_per_epoch = []
    for i in range(n_train // batch_size):

        # Training for one step.
        batch = mnist_data.train.next_batch(batch_size)    # Get one batch of training data.
        if i == 0:
            
            # Only for the first epoch, get the summary data.
            # Otherwise, it can clutter the visualization.
            l, _, gn_summ = session.run(
                [tf_loss, tf_loss_minimize, tf_gradnorm_summary], 
                feed_dict={train_inputs: batch[0].reshape(batch_size, image_size * image_size),
                           train_labels: batch[1],
                           tf_learning_rate: 0.01}
            )
            summ_writer.add_summary(gn_summ, epoch)
            
        else:
            
            # Optimize with training data.
            l, _ = session.run(
                [tf_loss, tf_loss_minimize],
                feed_dict={train_inputs: batch[0].reshape(batch_size, image_size * image_size),
                           train_labels: batch[1],
                           tf_learning_rate: 0.01}
            )
        
        loss_per_epoch.append(l)

    print("Average loss in epoch %d: %.5f" %(epoch, np.mean(loss_per_epoch)))    
    avg_loss = np.mean(loss_per_epoch)

    # Calculate the Validation Accuracy.
    valid_accuracy_per_epoch = []

    for i in range(n_valid // batch_size):
        valid_images, valid_labels = mnist_data.validation.next_batch(batch_size)
        valid_batch_predictions = session.run(
            tf_predictions, 
            feed_dict={train_inputs: valid_images.reshape(batch_size,image_size*image_size)}
        )
        valid_accuracy_per_epoch.append(accuracy(valid_batch_predictions, valid_labels))

    mean_v_acc = np.mean(valid_accuracy_per_epoch)
    print("\tAverage Valid Accuracy in epoch %d: %.5f"%(epoch, 
                                                        np.mean(valid_accuracy_per_epoch))
         )

    # Calculate the Test Accuracy.
    accuracy_per_epoch = []
    
    for i in range(n_test // batch_size):
        test_images, test_labels = mnist_data.test.next_batch(batch_size)
        test_batch_predictions = session.run(
            tf_predictions, 
            feed_dict={train_inputs: test_images.reshape(batch_size,image_size*image_size)}
        )
        accuracy_per_epoch.append(accuracy(test_batch_predictions, test_labels))

    print("\tAverage Test Accuracy in epoch %d: %.5f\n" %(epoch, np.mean(accuracy_per_epoch)))
    avg_test_accuracy = np.mean(accuracy_per_epoch)

    # Execute the summaries defined above.
    summ = session.run(performance_summaries, 
                       feed_dict={tf_loss_ph: avg_loss, 
                                  tf_accuracy_ph: avg_test_accuracy}
                      )

    # Write the obtained summaries to the file, so it can be displayed in the TensorBoard.
    summ_writer.add_summary(summ, epoch)

session.close()

W1006 21:04:13.190649 139798023042880 deprecation.py:323] From <ipython-input-6-d2776f5ada22>:26: read_data_sets (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
W1006 21:04:13.191390 139798023042880 deprecation.py:323] From /home/alex/anaconda3/envs/machine36/lib/python3.6/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:260: maybe_download (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
Instructions for updating:
Please write your own downloading logic.
W1006 21:04:13.192345 139798023042880 deprecation.py:323] From /home/alex/anaconda3/envs/machine36/lib/python3.6/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:262: extract_images (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated a

Extracting MNIST_data/train-images-idx3-ubyte.gz


W1006 21:04:13.481658 139798023042880 deprecation.py:323] From /home/alex/anaconda3/envs/machine36/lib/python3.6/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:267: extract_labels (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.data to implement this functionality.
W1006 21:04:13.483089 139798023042880 deprecation.py:323] From /home/alex/anaconda3/envs/machine36/lib/python3.6/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:110: dense_to_one_hot (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.one_hot on tensors.
W1006 21:04:13.535663 139798023042880 deprecation.py:323] From /home/alex/anaconda3/envs/machine36/lib/python3.6/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:290: DataSet.__init__ (from tensorflow.

Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Average loss in epoch 0: 0.99093
	Average Valid Accuracy in epoch 0: 92.76000
	Average Test Accuracy in epoch 0: 91.68000

Average loss in epoch 1: 0.18361
	Average Valid Accuracy in epoch 1: 95.94000
	Average Test Accuracy in epoch 1: 95.66000

Average loss in epoch 2: 0.11123
	Average Valid Accuracy in epoch 2: 96.72000
	Average Test Accuracy in epoch 2: 96.23000

Average loss in epoch 3: 0.08206
	Average Valid Accuracy in epoch 3: 97.22000
	Average Test Accuracy in epoch 3: 96.84000

Average loss in epoch 4: 0.06079
	Average Valid Accuracy in epoch 4: 97.48000
	Average Test Accuracy in epoch 4: 97.39000

Average loss in epoch 5: 0.04403
	Average Valid Accuracy in epoch 5: 97.68000
	Average Test Accuracy in epoch 5: 97.54000

Average loss in epoch 6: 0.03241
	Average Valid Accuracy in epoch 6: 97.36000
	Average Test Accuracy in epoch 6: 97.0

## Visualize the Computational Graph

![img](./figures/tensorboard_tutorial.png)

## Visualize the Summary Data

## Beyond Scalars: Visualizing Histograms/Distributions

## Defining Histogram Summaries to Visualize Weights and Biases

In [7]:
# Summaries need to be displayed.
# Create a summary for each weight and each bias in each layer.
all_summaries = []

for lid in layer_ids:
    with tf.name_scope(lid + "_hist"):
        with tf.variable_scope(lid, reuse=True):
            w, b = tf.get_variable("weights"), tf.get_variable("bias")

            # Create a scalar summary object for the loss so it can be displayed.
            tf_w_hist = tf.summary.histogram("weights_hist", tf.reshape(w, [-1]))
            tf_b_hist = tf.summary.histogram("bias_hist", b)
            all_summaries.extend([tf_w_hist, tf_b_hist])

# Merge all parameter histogram summaries together.
tf_param_summaries = tf.summary.merge(all_summaries)

## Executing the neural network (with Histogram Summaries)

In [8]:
image_size = 28
n_channels = 1
n_classes = 10
n_train = 55000
n_valid = 5000
n_test = 10000
n_epochs = 25

config = tf.ConfigProto(allow_soft_placement=True)
config.gpu_options.allow_growth = True
config.gpu_options.per_process_gpu_memory_fraction = 0.9 # Making sure Tensorflow doesn't 
                                                         # overflow the GPU.
session = tf.InteractiveSession(config=config)

if not os.path.exists("summaries"):
    os.mkdir("summaries")
if not os.path.exists(os.path.join("summaries", "third")):
    os.mkdir(os.path.join("summaries", "third"))

summ_writer_3 = tf.summary.FileWriter(os.path.join("summaries", "third"), session.graph)

tf.global_variables_initializer().run()

accuracy_per_epoch = []
mnist_data = input_data.read_data_sets("MNIST_data", one_hot=True)

for epoch in range(n_epochs):
    loss_per_epoch = []
    for i in range(n_train//batch_size):

        # Training for one step.
        batch = mnist_data.train.next_batch(batch_size) # Get one batch of training data.
        if i == 0:
            # Only for the first epoch, get the summary data.
            # Otherwise, it can clutter the visualization.
            l, _, gn_summ, wb_summ = \
                session.run([tf_loss, 
                             tf_loss_minimize,
                             tf_gradnorm_summary, 
                             tf_param_summaries],
                    feed_dict={train_inputs: batch[0].reshape(batch_size,image_size*image_size),
                               train_labels: batch[1],
                               tf_learning_rate: 0.00001}
                           )
            summ_writer_3.add_summary(gn_summ, epoch)
            summ_writer_3.add_summary(wb_summ, epoch)
        else:
            # Optimize with training data.
            l, _ = session.run([tf_loss, 
                               tf_loss_minimize],
                              feed_dict={train_inputs: batch[0].\
                                         reshape(batch_size, image_size * image_size),
                                         train_labels: batch[1],
                                         tf_learning_rate: 0.01}
                             )
        loss_per_epoch.append(l)

    print("Average loss in epoch %d: %.5f" %(epoch, np.mean(loss_per_epoch)))    
    avg_loss = np.mean(loss_per_epoch)

    # Calculate the Validation Accuracy.
    valid_accuracy_per_epoch = []
    for i in range(n_valid//batch_size):
        valid_images, valid_labels = mnist_data.validation.next_batch(batch_size)
        valid_batch_predictions = session.run(
            tf_predictions, feed_dict={train_inputs: valid_images.\
                                       reshape(batch_size,image_size * image_size)})
        valid_accuracy_per_epoch.append(
            accuracy(valid_batch_predictions,valid_labels)
        )

    mean_v_acc = np.mean(valid_accuracy_per_epoch)
    print("\tAverage Valid Accuracy in epoch %d: %.5f" \
          %(epoch,np.mean(valid_accuracy_per_epoch)))

    # Calculate the Test Accuracy.
    accuracy_per_epoch = []
    for i in range(n_test//batch_size):
        test_images, test_labels = mnist_data.test.next_batch(batch_size)
        test_batch_predictions = session.run(
            tf_predictions,feed_dict={train_inputs: test_images.\
                                      reshape(batch_size,image_size*image_size)}
        )
        accuracy_per_epoch.append(accuracy(test_batch_predictions,test_labels))

    print("\tAverage Test Accuracy in epoch %d: %.5f\n" %\
          (epoch,np.mean(accuracy_per_epoch)))
    avg_test_accuracy = np.mean(accuracy_per_epoch)

    # Execute the summaries defined above.
    summ = session.run(performance_summaries, 
                       feed_dict={tf_loss_ph:avg_loss, tf_accuracy_ph:avg_test_accuracy})

    # Write the obtained summaries to the file, so they can be displayed.
    summ_writer_3.add_summary(summ, epoch)

session.close()

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Average loss in epoch 0: 1.00752
	Average Valid Accuracy in epoch 0: 92.24000
	Average Test Accuracy in epoch 0: 92.06000

Average loss in epoch 1: 0.19359
	Average Valid Accuracy in epoch 1: 95.22000
	Average Test Accuracy in epoch 1: 95.32000

Average loss in epoch 2: 0.11883
	Average Valid Accuracy in epoch 2: 97.28000
	Average Test Accuracy in epoch 2: 96.96000

Average loss in epoch 3: 0.08186
	Average Valid Accuracy in epoch 3: 97.18000
	Average Test Accuracy in epoch 3: 97.08000

Average loss in epoch 4: 0.05932
	Average Valid Accuracy in epoch 4: 97.48000
	Average Test Accuracy in epoch 4: 97.23000

Average loss in epoch 5: 0.04555
	Average Valid Accuracy in epoch 5: 97.88000
	Average Test Accuracy in epoch 5: 97.63000

Average loss in epoch 6: 0.03505
	Average Valid Accuracy in epoch 6:

## Visualizing Histogram Data of Weights and Biases

## The Effect of Different Initializers

In [9]:
batch_size = 100
layer_ids = ["hidden1", "hidden2", "hidden3", "hidden4", "hidden5", "out"]
layer_sizes = [784, 500, 400, 300, 200, 100, 10]

tf.reset_default_graph()

# Inputs and Labels.
train_inputs = tf.placeholder(tf.float32, shape=[batch_size, layer_sizes[0]], 
                              name="train_inputs")
train_labels = tf.placeholder(tf.float32, shape=[batch_size, layer_sizes[-1]], 
                              name="train_labels")

# Weight and Bias definitions.
for idx, lid in enumerate(layer_ids):

    with tf.variable_scope(lid):
        w = tf.get_variable("weights", shape=[layer_sizes[idx], layer_sizes[idx+1]],
                            initializer=tf.contrib.layers.xavier_initializer())
        b = tf.get_variable("bias", shape=[layer_sizes[idx+1]],
                            initializer=tf.random_uniform_initializer(-0.1, 0.1))

## Calculating Logits, Predictions, Loss and Optimization

## Define Summaries

## Histogram Summaries: Visualizing Weights and Biases

## Execute the neural network

## How To Compare Different Initialization Techniques

## Distribution View of Histograms

## Observations and Conclusions